# Игра "5 Букв"
Тинькофф перезапустил игру "5 букв", в которой нужно отгадывать слова. Вам нужно набрать существительное из пяти букв, система покажет какие буквы этого слова есть в загаданном слове, на основе этой информации вы должны отгадать слово за несколько попыток.  
Для призов банк подготовил скидки, кэшбэки и др. Например, за 60 слов можно получить 50%-й кэшбэк в Ozon.
Играем в игру из приложения Тиньков банка "5 букв".  Правила игры похожи на игру Wordle.
<div>
<img src="game.jpeg" width="200"/>
</div>   
Попробуем придумать какую-нибудь стратегию для выигрыша.

## Чтение файла

Для нашей игры прочитаем список слов, которые:
* состоят из 5 букв
* не содержат символы не из алфавита (дефисы, английские буквы)
* не начинаются с заглавной буквы  

Исходный файл [nouns.csv](https://github.com/Badestrand/russian-dictionary/blob/master/nouns.csv) взят с гитхаба (https://github.com/Badestrand/russian-dictionary) (Creative Commons Attribution Share Alike 4.0).

In [1]:
words = []
with open('nouns.txt', encoding='utf-8') as file:
    words = [w.strip() for w in file.readlines()]

words = [w for w in words if not 'ё' in w] # отбрасываем слова с буквой "ё" 
words.sort()
(words[:10], len(words))

(['аарон',
  'абака',
  'аббат',
  'абвер',
  'абзац',
  'аборт',
  'абрек',
  'абрис',
  'абхаз',
  'абцуг'],
 3620)

## Класс WordPredictor

In [2]:
class WordPredictor:
    def __init__(self, words, takeplace=True):
        """
        words     : массив слов
        takeplace : Учитывать место буквы при подсчете рейтинга слова. 
                    Например встречаемость буквы й в зависимости от места: [0, 1, 38, 8, 50]
        """
        self.takeplace = takeplace
        self.words = words.copy() # TODO: проверка, что бы все слова были одинаковой длины
        self.alphabet = ''
        
        # для расчета рейтинга слова соберем статистику по буквам
        dict={}

        for w in self.words:
            for l in w:
                if not l in self.alphabet:
                    self.alphabet += l
                dict[l] = dict.get(l, 0) + 1
        
        sorted_tuples = sorted(dict.items(), key=lambda x: x[1],reverse=True)
        self.letter_rating = {k: v for k, v in sorted_tuples}
        mostusedletters = "".join(list(self.letter_rating.keys()))
        self.alphabet = ''.join(sorted(self.alphabet))
        print(f"Алфавит: {self.alphabet}")
        print(f"Буквы, отсортированные по степени встречаемости: {mostusedletters}")
        
        # а теперь посмотрим, на каких местах каждая буква встречается чаще
        self.letter_places={k: [0]*5 for k in self.alphabet}
        for w in self.words:
            for l,i in zip(w, range(5)):        
                self.letter_places[l][i] += 1
         
        self.reset()
        
                
    def score(self, w, ignore=''):
        """
        Подсчет рейтинга слова
        ignore     : Строка с метками для, определающая, учитывать букву при расчете рейтинга слова (символ "_") или нет.
                     Это нужно, что бы считать рейтинг слов, для которых уже угаданы позиции букв. 
                     В этом случае угаданные буквы игнорируются в рейтинге
        """
        if ignore == '': ignore = '_'*len(w) # если ignore не задана, то считаем рейтинг по всем буквам
        s=0
        for i in range(len(w)):
            if ignore[i] not in self.alphabet: # вместо буквы находится символ, значит буква учавствует в подсчете
                if not self.takeplace:
                    s += self.sorted_dict[w[i]]//w.count(w[i])
                else:
                    s += self.letter_places[w[i]][i]//w.count(w[i])
        return s
    
    def reset(self):
        """
        сброс попыток подбора слова
        """
        self.wrongletters=''               # список букв, которых вообще нет в угадываемом слове
        self.wrongplaces={}                # словарь букв, которые есть в слове, но не на неправильной позиции
        self.rightplaces='_'*len(words[0]) # список угаданных букв
        self.attempts = []                 # список попыток с результатом
        self.goodwords = words.copy()      # список слов, которые удовлетворяют условиям
        self.endgame = False               # флаг окончания поиска
        
    
    def scan_results(self, testword, result):
        """
        Разбор результата проверки с записью в соответствующие переменные
        testword : слово, которое проверялось
        result   : результат проверки, "-" - буквы в слове нет, "*" - буква есть, но не на другом месте
                   "+" - буква угадана
        """
        # TODO: проверка на равенство длин testword, result и words
        for i, r in enumerate(result):
            l = testword[i]
            if (r=='-'): 
                self.wrongletters = ''.join(set(self.wrongletters) | set(l))
            elif (r=='*'):
                self.wrongplaces[l] = self.wrongplaces.get(l,[]) + [i]
            else:
                t = list(self.rightplaces)
                t[i] = l
                self.rightplaces = ''.join(t)
                

    def is_good_word(self, w):
        """
        проверка слова по условиям wrongletters, wrongplaces, rightplaces
        """
        
        # 1) в слове должны быть найденные буквы в нужных местах
        for testletter, goodletter in zip(w, self.rightplaces):
            if goodletter != '_': # если угаданная буква есть на этом месте
                if testletter != goodletter: # и она не равна проверяемой букве
                    return False # сразу нет

        # 2) в слове не должно быть отсутствующих букв
        for testletter in w:
            if testletter in self.wrongletters:
                return False

        # 3) в слове должны быть неправильно расположенные буквы
        for goodletter in list(self.wrongplaces.keys()): # берем каждую неправильно расположенную букву
            if goodletter not in w: # неправильно расположенные буквы должны быть в слове (странно, да?)
                return False
            # 4) в слове неправильно расположенные буквы не должны быть на неправильных местах
            else: # если неправильно расположенная буква есть в проверяемом слове
                if w.index(goodletter) in self.wrongplaces[goodletter]: # место буквы в слове есть в списке неправильных мест
                    return False
                
        return True # если слово прошло все проверки

    def find_good_words(self):
        """
        Вывод списка слов, удовлетворяющих условиям в переменных wrongletters, wrongplaces, rightplaces
        """
        goodwordsnext = []

        for w in self.goodwords:
            if self.is_good_word(w):
                goodwordsnext += [w]
        return goodwordsnext
        
    def predict_next_words(self, testword, result):
        """
        главная функция предсказания следующих слов
        testword  :  слово, которое было указано в приложении
        result    :  результат предсказания этого слова
        """
        if self.endgame:
            print('Нужно сбросить поиск вызвав reset()')
            return None, None
        
        self.attempts.append([testword, result]) # добавляем попытку в массив попыток (зачем?)
        self.scan_results(testword, result)      # сканируем результат попытки
        nextwords = self.find_good_words()  # ищем слова, подходящие под условия
        
        if len(nextwords) == 0:
            print('Что-то пошло не так. Не могу найти слова, попадающие под условия')
            self.endgame = True
            return None, (None, None)
        elif len(nextwords) == 1:
            # print(f'Слово найдено: {nextwords[0]}.\nЧисло ходов {len(self.attempts)}.')
            self.endgame = True
            return "1 из 1:", [[nextwords[0], self.score(nextwords[0])]]
        else:
            self.goodwords = nextwords
            scorelist=[[w, self.score(w, self.rightplaces)] for w in self.goodwords] # считаем рейтинг слов...
            newwords=sorted(scorelist,key=lambda x: x[1],reverse=True) # и сортируем по убыванию
            return f"{min(10, len(newwords))} из {len(newwords)}:", newwords[:10]
    
    @staticmethod
    def check_word(rightword, testword):
        """
        проверка слова, как это присходит в игре
        """
        result=''
        for rl, tl in zip(rightword, testword):
            if tl not in rightword:
                result = result + '-'
            else:
                if tl==rl:
                    result = result + '+'
                else:
                    result = result + '*'
        return result
    

    def find_word(self, secret, tryword='порка'):
        """
        поиск загаданного слова secret, используюя описанные в классе алгоритмы
        """
        self.reset()
    
        i = 0
        while i < 10:
            i += 1
            checkresult = WordPredictor.check_word(secret, tryword) # проверяем слово
            _, nextwords = self.predict_next_words(tryword, checkresult) # ищем новые слова
            print( (tryword, checkresult, self.rightplaces, len(nextwords)) )
            if(len(nextwords)==1):
                print(f'Конец игры. Загаданное слово: {nextwords[0][0]}')
                break
            else:
                tryword=nextwords[0][0]

## Проверка алгоритма поиска

In [3]:
wp = WordPredictor(words)

Алфавит: абвгдежзийклмнопрстуфхцчшщъыьэюя
Буквы, отсортированные по степени встречаемости: аокриетнлсупмбвдгзяшьыхчфжйцющэъ


In [4]:
wp.reset()
wp.find_word('чайка')

('порка', '---++', '___ка', 10)
('силка', '---++', '___ка', 10)
('метка', '---++', '___ка', 10)
('банка', '-+-++', '_а_ка', 10)
('кадка', '*+-++', '_а_ка', 8)
('шавка', '-+-++', '_а_ка', 4)
('гайка', '-++++', '_айка', 2)
('зайка', '-++++', '_айка', 1)
Конец игры. Загаданное слово: чайка


## Пример поиска слова "манеж"

<div>
<img src="example.png" width="200"/>  
</div>  
1) Начинаем со слова "порка":

In [5]:
wp.reset() # сбрасываем поиск
wp.predict_next_words('порка', '----*')

('10 из 372:',
 [['сатин', 1977],
  ['салун', 1853],
  ['балет', 1844],
  ['валет', 1826],
  ['ватин', 1815],
  ['талия', 1786],
  ['бадин', 1754],
  ['талес', 1750],
  ['баден', 1711],
  ['шатен', 1701]])

2\) Пусть следующее слово будет "сатин"

In [6]:
wp.predict_next_words('сатин', '-+--*')

('10 из 31:',
 [['банда', 931],
  ['майна', 916],
  ['манна', 851],
  ['ганза', 840],
  ['ванна', 835],
  ['фауна', 817],
  ['фанза', 801],
  ['манеж', 790],
  ['ганец', 774],
  ['ханша', 760]])

3\) Более менее под лексикон игры подходят "фауна" и "манеж", так что пусть следующее слово "манеж", и оно оказывается верным

In [7]:
wp.reset() # сбрасываем поиск
wp.predict_next_words('порка', '-*---')

('10 из 200:',
 [['силон', 1788],
  ['сулой', 1630],
  ['сезон', 1629],
  ['бетон', 1593],
  ['бутон', 1583],
  ['мутон', 1581],
  ['фелон', 1579],
  ['мелос', 1563],
  ['зелот', 1543],
  ['нилот', 1516]])

In [8]:
wp.predict_next_words('бетон', '---')

('10 из 64:',
 [['силон', 1788],
  ['сулой', 1630],
  ['сифон', 1516],
  ['синод', 1501],
  ['лимон', 1451],
  ['силос', 1413],
  ['шифон', 1283],
  ['судно', 1219],
  ['вылом', 1184],
  ['выгон', 1177]])